In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
from matplotlib.pyplot import subplots, style, rc
from tqdm import tqdm
from collections import defaultdict
from itertools import count, islice
from functools import lru_cache

In [3]:
style.use(["seaborn-poster", "seaborn-whitegrid"])
rc("axes", linewidth=1, edgecolor="black")
%matplotlib inline

In [19]:
h10 = pd.read_csv("HG002.10kb-names.tsv", sep="\t", names=["name", "chrom"])
h10["sample"] = "HG002.10kb"
h15 = pd.read_csv("HG002.15kb-names.tsv", sep="\t", names=["name", "chrom"])
h15["sample"] = "HG002.15kb"
sample_assignment = pd.concat([h10, h15], axis=0)

In [47]:
from glob import glob

kmerscans = []
for dat in glob("HG002.10kb+15kb-levenshtein-q_arm/*dat.gz"):
    ks = pd.read_csv(dat, sep="\t", escapechar="#", usecols=(0, 2))
    ks["haplotype"] = ks["chrom"].apply(lambda c: c.split(":haplotype ")[1])
    ks["chrom"] = ks["chrom"].apply(lambda c: c.split(":haplotype ")[0])
    kmerscans.append(ks)

kmerscan = pd.concat(kmerscans, axis=0).drop_duplicates()

In [48]:
pd.merge(kmerscan, sample_assignment)[:3]

,name,chrom,haplotype,sample
0,m54238_180701_114913/17892208/ccs,18qtel_1-500K_1_12_12_rc,2,HG002.10kb
1,m54316_180719_115850/24379831/ccs,18qtel_1-500K_1_12_12_rc,2,HG002.10kb
2,m54315_180714_142044/68878424/ccs,18qtel_1-500K_1_12_12_rc,1,HG002.10kb


In [50]:
assigner = pd.merge(kmerscan, sample_assignment).drop(columns="name")
assigner["count"] = 1
calculator = assigner.groupby(["chrom", "haplotype", "sample"], as_index=False).sum()
h1 = calculator.query("haplotype=='1'").pivot(index="chrom", columns="sample", values="count").reset_index()
h1["haplotype"] = 1
h2 = calculator.query("haplotype=='2'").pivot(index="chrom", columns="sample", values="count").reset_index()
h2["haplotype"] = 2

assignment = pd.concat([h1, h2], axis=0).fillna(0)
assignment.columns.name = None
assignment["HG002.10kb"] = assignment["HG002.10kb"].astype(int)
assignment["HG002.15kb"] = assignment["HG002.15kb"].astype(int)
assignment = assignment.sort_values(by=["chrom", "haplotype"]).iloc[:,[0,3,1,2]]
assignment.to_csv("HG002.10kb+15kb-haplotype-assignment.tsv", sep="\t", index=False)